In [1]:
import cv2
import numpy as np
import time
# import mss
# import mss.tools #utilisé pour capturer l'écran

ModuleNotFoundError: No module named 'mss'

In [19]:
vid = cv2.VideoCapture("test_videos/test-compass3.mp4")

In [20]:
""""#Lire la vidéo
#Appuyer sur q pour fermer la fenetre
while True:
    ret, frame = vid.read() #Lire une image de la video
    if not ret:
        break
    cv2.imshow("Vidéo",frame) #On affiche l'image
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
vid.release()
cv2.destroyAllWindows()"""

'"#Lire la vidéo\n#Appuyer sur q pour fermer la fenetre\nwhile True:\n    ret, frame = vid.read() #Lire une image de la video\n    if not ret:\n        break\n    cv2.imshow("Vidéo",frame) #On affiche l\'image\n    if cv2.waitKey(1) & 0xFF == ord(\'q\'):\n        break\nvid.release()\ncv2.destroyAllWindows()'

In [21]:
#Detection sur vidéo
template = cv2.imread("compass_template_center.jpg", cv2.IMREAD_GRAYSCALE)
h, w = template.shape[:2]
fps = int(vid.get(cv2.CAP_PROP_FPS)) #Fréquence d'image imposée
print(f"FPS de la vidéo d'origine : {fps}")
frame_time = 1 / fps
start_time = time.time() #temps de départ
frame_id = 0
while vid.isOpened():
    ret, frame = vid.read() #Lire une image de la video
    if not ret:
        break
        
    frame_id += 1
    # On traite la vidéo toute les X frames 
    if frame_id % 2 == 0:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #On passe en niveau de gris (obligatoire avec matchTemplate)
        result = cv2.matchTemplate(gray, template, cv2.TM_CCOEFF_NORMED) #Matching
        threshold = 0.9 #en dessous de 0.8 bcp de faux positifs
        loc = np.where(result >= threshold) #Bonne correspondance
        print(f"Template détecté à: {loc}")
        # Dessiner des rectangles autour des correspondances
    else:
        loc = []
    for pt in zip(*loc[::-1]): #Coordonnées coins superieur gauche
        bottom_right = (pt[0] + w, pt[1] + h) #Bas à droite
        cv2.rectangle(frame, pt, bottom_right, (0, 255, 0), 2)
    cv2.namedWindow("Detection", cv2.WINDOW_NORMAL) #Permet de redimensionner la fenetre et de voir toute la vidéo (sinon image tronquée)
    cv2.imshow("Detection",frame) #On affiche l'image 
    if cv2.waitKey(int(1)) & 0xFF == ord('q'):
        break
    
vid.release()
cv2.destroyAllWindows()



FPS de la vidéo d'origine : 30
Template détecté à: (array([ 71,  72, 742]), array([837, 837, 128]))
Template détecté à: (array([ 71,  72, 742]), array([836, 836, 128]))
Template détecté à: (array([ 71,  72, 742]), array([835, 835, 128]))
Template détecté à: (array([ 71,  72, 742]), array([834, 834, 128]))
Template détecté à: (array([ 71,  72, 742]), array([833, 833, 128]))
Template détecté à: (array([ 71,  72, 742]), array([826, 826, 128]))
Template détecté à: (array([ 71,  72, 742]), array([827, 827, 128]))
Template détecté à: (array([ 71,  72, 742]), array([830, 830, 128]))
Template détecté à: (array([ 71,  72, 742]), array([836, 836, 128]))
Template détecté à: (array([ 71,  72, 742]), array([845, 845, 128]))
Template détecté à: (array([ 71,  72, 742]), array([855, 855, 128]))
Template détecté à: (array([ 71,  72, 742]), array([871, 871, 128]))
Template détecté à: (array([ 71,  72, 742]), array([894, 894, 128]))
Template détecté à: (array([ 71,  72, 742]), array([927, 927, 128]))
Tem

In [33]:
#Detection en temps reel
vid = cv2.VideoCapture("test_videos/test-compass3.mp4")
template = cv2.imread("compass_template_center.jpg", cv2.IMREAD_GRAYSCALE)
h, w = template.shape[:2]
fps = int(vid.get(cv2.CAP_PROP_FPS)) #Fréquence d'image imposée
print(f"FPS de la vidéo d'origine : {fps}")
frame_time = 1 / fps
start_time = time.time() #temps de départ
frame_id = 0
# Définir la zone de capture (adapter selon la résolution et la position du jeu)
mon = {'top': 100, 'left': 100, 'width': 800, 'height': 600}
with mss.mss() as sct:
#while vid.isOpened():
    while True:
        screenshot = sct.grab(mon)  # Capture de l'écran
        screen = np.array(screenshot)  # Conversion en tableau numpy
        screen = cv2.cvtColor(screen, cv2.COLOR_BGRA2BGR) #suppression canal alpha
        ret, frame = vid.read() #Lire une image de la video
        if not ret:
            break
            
        frame_id += 1
        # On traite la vidéo toute les X frames 
        if frame_id % 1 == 0:
            gray = cv2.cvtColor(screen, cv2.COLOR_BGR2GRAY) #On passe en niveau de gris (obligatoire avec matchTemplate)
            result = cv2.matchTemplate(gray, template, cv2.TM_CCOEFF_NORMED) #Matching
            threshold = 0.8 #en dessous de 0.8 bcp de faux positifs
            loc = np.where(result >= threshold) #Bonne correspondance
            #print(f"Template détecté à: {loc}")
            # Dessiner des rectangles autour des correspondances
        else:
            loc = []
        for pt in zip(*loc[::-1]):#Coordonnées coins superieur gauche
            #print(f"Coordonnées détectée :", pt)
            x, y = int(pt[0]), int(pt[1])
            bottom_right = (x + w, y + h) #Bas à droite
            cv2.rectangle(frame, (x,y), bottom_right, (0, 255, 0), 2)
        cv2.namedWindow("Detection", cv2.WINDOW_NORMAL) #Permet de redimensionner la fenetre et de voir toute la vidéo (sinon image tronquée)
        print(frame.shape)
        cv2.imshow("Detection",screen) #On affiche l'image 
        if cv2.waitKey(int(1)) & 0xFF == ord('q'):
            break
        
vid.release()
cv2.destroyAllWindows()


FPS de la vidéo d'origine : 30
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 19